In [1]:
import pandas as pd

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset cnn_dailymail (/Users/rcgreen/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00, 50.56it/s]

Features: ['article', 'highlights', 'id']


In [3]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}): """)
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):') 
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051): 
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [4]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

/var/folders/1n/hpwh_ls53hdd_n63d85jf67w0000gn/T/ipykernel_2786/841839976.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


In [5]:
# def evaluate_summaries_baseline(dataset, metric, column_text="article",
#     column_summary="highlights"):
#     summaries = [three_sentence_summary(text) for text in dataset[column_text]]
#     metric.add_batch(predictions=summaries, references=dataset[column_summary])
#     score = metric.compute() 
#     return score

In [6]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000))
# score = evaluate_summaries_baseline(test_sampled, rouge_metric) 
# rouge_dict = dict((rn, score[rn].mid.fmeasure) 
#                   for rn in rouge_names) 
# pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

In [7]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=16, device=device,
                                   column_text="article",
                                   column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size)) 
    
    for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                             attention_mask=inputs["attention_mask"].to(device),
                             length_penalty=0.8, num_beams=8, max_length=128)
            
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                                    for s in summaries]
            
            decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries] 
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute() 
    return score


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=8) 
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])